## Importacion de librerias

In [71]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from statistics import mean

## Definicion de funciones

## Obtencion y procesamiento de los datos

In [72]:
data = pd.read_csv('Dataset/iris.data', header = None)
data.columns = ['Sepal length','Sepal width','Petal length','Petal width','Class']
# No tiene datos vacios, ni NaN, ni repetidos
data

,Sepal length,Sepal width,Petal length,Petal width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [73]:
array = np.array([144, 143, 141, 137, 130, 122, 117, 111, 107, 102,
                  96, 92, 87, 83, 80, 75, 69, 64, 58, 55,
                  44, 37, 30, 28, 22, 20, 16, 10, 8, 3
                ])

dataV = pd.DataFrame()

for elem in array:
    dataV = dataV.append(data.iloc[elem])
    data = data.drop(elem, axis = 0)

data = data.reset_index(drop = True)
dataV = dataV.reset_index(drop = True)

In [74]:
dataX = data.drop('Class', axis = 1)
dataY = data['Class']

dataY = dataY.map({
                'Iris-setosa':0,
                'Iris-versicolor':1,
                'Iris-virginica':2,
                },
     na_action = None)

dataXV = dataV.drop('Class', axis = 1)
dataYV = dataV['Class']

dataYV = dataYV.map({
                'Iris-setosa':0,
                'Iris-versicolor':1,
                'Iris-virginica':2,
                },
     na_action = None)

In [75]:
dataYC = to_categorical(dataY, 3) # 3 = numClasses
dataYVC = to_categorical(dataYV, 3)
#dataYC = dataY/2

In [76]:
maximosTrain = dataX.max()
maximosVal = dataXV.max()

# Normalizacion de datos (se dividen todos los valores de cada atributo entre el mayor de ese atributo)
dataX.iloc[:, :] = dataX.iloc[:, :]/maximosTrain
dataXV.iloc[:, :] = dataXV.iloc[:, :]/maximosVal

## Creacion del perceptron

In [77]:
def crearModelo(dataX, dataYC, dataXV, dataYVC,
                neuronasOculta, neuronasSalida, activacionOculta, activacionSalida,
                epochs, batch_size, loss):
    
    input_shape = (4,)

    """ ARQUITECTURA DEL MODELO """

    # Se crea un modelo secuencial
    model = Sequential()

    """
                150/(2*4) < n < (2*150)/4   ---->> 150/8 < n < 300/4  ---->> 18,75 < n < 75

    """
    
    print(f"MODELO:\n\t- Capa oculta ({neuronasOculta}, {activacionOculta})"
         f"\n\t- Capa salida ({neuronasSalida}, {activacionSalida})"
         f"\n\t- Epochs: {epochs}\n\t- Batch_size: {batch_size}"
          f"\n\t- Loss metric: {loss}\n\n")

    # Primera capa oculta, (tambien se define la capa de entrada con input_shape)
    model.add(Dense(neuronasOculta, input_shape = input_shape,
                    activation = activacionOculta))


    # Capa de salida, el 2 (numero de neuronas de la capa) es la cantidad de salidas posibles que puede dar (clases en las que clasificar)
    model.add(Dense(neuronasSalida, activation = activacionSalida))

    # Configuracion del modelo
    model.compile(loss = loss, # Funcion de error 
                  optimizer='adam', # Modificacion matriz de pesos
                  metrics=['accuracy'])

    # Se entrena el modelo
    # Batch_size = 105 para partir el dataset de entrenamiento en 4 porciones para cada iteracion
    model.fit(dataX, dataYC, epochs = epochs, batch_size = batch_size, verbose = True)
    
    
    resultadosValidacion = model.evaluate(dataXV, dataYVC, verbose = 1)

    print(f"\nResultados de la validacion\n\t- Error: {resultadosValidacion[0]}\n\t- Precision: {resultadosValidacion[1]*100}%\n\n")
    
    return [model, resultadosValidacion]

In [78]:
modeloEjemplo = crearModelo(dataX, dataYC, dataXV, dataYVC, 20, 3, 'relu', 'softmax', 
                            20, 30, 'categorical_crossentropy')

MODELO:
	- Capa oculta (20, relu)
	- Capa salida (3, softmax)
	- Epochs: 20
	- Batch_size: 30
	- Loss metric: categorical_crossentropy


Epoch 1/20
4/4 [==============================] - 0s 2ms/step - loss: 1.1555 - accuracy: 0.0333
Epoch 2/20
4/4 [==============================] - 0s 2ms/step - loss: 1.1432 - accuracy: 0.0417
Epoch 3/20
4/4 [==============================] - 0s 2ms/step - loss: 1.1313 - accuracy: 0.0417
Epoch 4/20
4/4 [==============================] - 0s 2ms/step - loss: 1.1200 - accuracy: 0.0417
Epoch 5/20
4/4 [==============================] - 0s 2ms/step - loss: 1.1088 - accuracy: 0.0583
Epoch 6/20
4/4 [==============================] - 0s 2ms/step - loss: 1.0979 - accuracy: 0.1083
Epoch 7/20
4/4 [==============================] - 0s 2ms/step - loss: 1.0877 - accuracy: 0.2000
Epoch 8/20
4/4 [==============================] - 0s 1ms/step - loss: 1.0779 - accuracy: 0.3750
Epoch 9/20
4/4 [==============================] - 0s 2ms/step - loss: 1.0694 - accuracy: 0.4167

In [79]:
neuronasOcultaModelos = [19, 19, 19, 19, 19, 19]
neuronasSalidaModelos = [3, 3, 3, 3, 3, 3]
activacionesOcultaModelos = ['relu', 'relu', 'relu', 'sigmoid', 'sigmoid', 'sigmoid']
activacionesSalidaModelos = ['softmax', 'softmax', 'softmax', 'softmax', 'softmax', 'softmax']
epochsModelos = [15, 20, 25, 15, 20, 25]
batchSizeModelos = [30, 30, 30, 30, 30, 30]

In [80]:
lossesModelos = ['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy', 
                   'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy']

listaModelos = []

for i in range(0, len(neuronasOcultaModelos)):
    
    print(f"MODELO NUMERO {i+1}:\n")
    
    listaModelos.append(crearModelo(dataX, dataYC, dataXV, dataYVC, neuronasOcultaModelos[i], neuronasSalidaModelos[i],
               activacionesOcultaModelos[i], activacionesSalidaModelos[i], 
               epochsModelos[i], batchSizeModelos[i], lossesModelos[i]))

MODELO NUMERO 1:

MODELO:
	- Capa oculta (19, relu)
	- Capa salida (3, softmax)
	- Epochs: 15
	- Batch_size: 30
	- Loss metric: categorical_crossentropy


Epoch 1/15
4/4 [==============================] - 0s 1ms/step - loss: 1.1257 - accuracy: 0.3333
Epoch 2/15
4/4 [==============================] - 0s 2ms/step - loss: 1.1150 - accuracy: 0.3333
Epoch 3/15
4/4 [==============================] - 0s 1ms/step - loss: 1.1059 - accuracy: 0.3333
Epoch 4/15
4/4 [==============================] - 0s 1ms/step - loss: 1.0965 - accuracy: 0.3333
Epoch 5/15
4/4 [==============================] - 0s 1ms/step - loss: 1.0884 - accuracy: 0.3333
Epoch 6/15
4/4 [==============================] - 0s 1ms/step - loss: 1.0798 - accuracy: 0.3333
Epoch 7/15
4/4 [==============================] - 0s 1ms/step - loss: 1.0727 - accuracy: 0.3333
Epoch 8/15
4/4 [==============================] - 0s 2ms/step - loss: 1.0652 - accuracy: 0.3333
Epoch 9/15
4/4 [==============================] - 0s 1ms/step - loss: 1.0586 

4/4 [==============================] - 0s 1ms/step - loss: 1.1338 - accuracy: 0.3333
Epoch 14/15
4/4 [==============================] - 0s 1ms/step - loss: 1.1306 - accuracy: 0.3333
Epoch 15/15
1/1 [==============================] - 0s 87ms/step - loss: 1.1295 - accuracy: 0.3333

Resultados de la validacion
	- Error: 1.1294761896133423
	- Precision: 33.33333432674408%


MODELO NUMERO 5:

MODELO:
	- Capa oculta (19, sigmoid)
	- Capa salida (3, softmax)
	- Epochs: 20
	- Batch_size: 30
	- Loss metric: categorical_crossentropy


Epoch 1/20
4/4 [==============================] - 0s 1ms/step - loss: 1.1209 - accuracy: 0.3333
Epoch 2/20
4/4 [==============================] - 0s 1ms/step - loss: 1.1123 - accuracy: 0.3333
Epoch 3/20
4/4 [==============================] - 0s 2ms/step - loss: 1.1065 - accuracy: 0.3333
Epoch 4/20
4/4 [==============================] - 0s 1ms/step - loss: 1.1026 - accuracy: 0.3333
Epoch 5/20
4/4 [==============================] - 0s 1ms/step - loss: 1.0985 - accura

In [81]:
bestLoss = 0
bestAcc = 0
bestPosition = 0
position = 0

for model in listaModelos:
    print(f"- VALIDATION LOSS: {model[1][0]}\n- VALIDATION ACCURACY: {model[1][1]}\n\n")
    
    if model[1][1] > bestAcc:
        bestAcc = model[1][1]
        bestPosition = position
    
    position += 1
    
listaModelos[bestPosition][0].save('bestModelCategoricalCrossentropy', save_format = 'tf') # save_f = h5
# ('bestModelCategoricalCrossentropy.h5')
listaModelos[bestPosition][0].save_weights('bestModelCategoricalCrossentropyWeights', save_format = 'tf')
#listaModelos[bestPosition][0].save_weights('bestModelCategoricalCrossentropyWeights.keras')

- VALIDATION LOSS: 1.0189132690429688
- VALIDATION ACCURACY: 0.1666666716337204


- VALIDATION LOSS: 0.9945383667945862
- VALIDATION ACCURACY: 0.6666666865348816


- VALIDATION LOSS: 1.0972827672958374
- VALIDATION ACCURACY: 0.4333333373069763


- VALIDATION LOSS: 1.1294761896133423
- VALIDATION ACCURACY: 0.3333333432674408


- VALIDATION LOSS: 1.0588501691818237
- VALIDATION ACCURACY: 0.6666666865348816


- VALIDATION LOSS: 1.033536434173584
- VALIDATION ACCURACY: 0.800000011920929


INFO:tensorflow:Assets written to: bestModelCategoricalCrossentropy\assets


In [82]:
lossesModelos = ['mean_squared_error', 'mean_squared_error', 'mean_squared_error', 
                   'mean_squared_error', 'mean_squared_error', 'mean_squared_error']

listaModelos = []

for i in range(0, len(neuronasOcultaModelos)):
    
    print(f"MODELO NUMERO {i+1}:\n")
    
    listaModelos.append(crearModelo(dataX, dataYC, dataXV, dataYVC, neuronasOcultaModelos[i], neuronasSalidaModelos[i],
               activacionesOcultaModelos[i], activacionesSalidaModelos[i], 
               epochsModelos[i], batchSizeModelos[i], lossesModelos[i]))

MODELO NUMERO 1:

MODELO:
	- Capa oculta (19, relu)
	- Capa salida (3, softmax)
	- Epochs: 15
	- Batch_size: 30
	- Loss metric: mean_squared_error


Epoch 1/15
4/4 [==============================] - 0s 1ms/step - loss: 0.2131 - accuracy: 0.6833
Epoch 2/15
4/4 [==============================] - 0s 1ms/step - loss: 0.2111 - accuracy: 0.6667
Epoch 3/15
4/4 [==============================] - 0s 2ms/step - loss: 0.2094 - accuracy: 0.6667
Epoch 4/15
4/4 [==============================] - 0s 1ms/step - loss: 0.2078 - accuracy: 0.6667
Epoch 5/15
4/4 [==============================] - 0s 2ms/step - loss: 0.2063 - accuracy: 0.6667
Epoch 6/15
4/4 [==============================] - 0s 2ms/step - loss: 0.2048 - accuracy: 0.6667
Epoch 7/15
4/4 [==============================] - 0s 1ms/step - loss: 0.2033 - accuracy: 0.6667
Epoch 8/15
4/4 [==============================] - 0s 2ms/step - loss: 0.2019 - accuracy: 0.6667
Epoch 9/15
4/4 [==============================] - 0s 2ms/step - loss: 0.2005 - accu

4/4 [==============================] - 0s 1ms/step - loss: 0.2149 - accuracy: 0.4917
Epoch 15/15
1/1 [==============================] - 0s 72ms/step - loss: 0.2134 - accuracy: 0.4000

Resultados de la validacion
	- Error: 0.21336695551872253
	- Precision: 40.00000059604645%


MODELO NUMERO 5:

MODELO:
	- Capa oculta (19, sigmoid)
	- Capa salida (3, softmax)
	- Epochs: 20
	- Batch_size: 30
	- Loss metric: mean_squared_error


Epoch 1/20
4/4 [==============================] - 0s 1ms/step - loss: 0.2209 - accuracy: 0.3333
Epoch 2/20
4/4 [==============================] - 0s 2ms/step - loss: 0.2197 - accuracy: 0.3333
Epoch 3/20
4/4 [==============================] - 0s 1ms/step - loss: 0.2188 - accuracy: 0.3333
Epoch 4/20
4/4 [==============================] - 0s 2ms/step - loss: 0.2181 - accuracy: 0.5417
Epoch 5/20
4/4 [==============================] - 0s 1000us/step - loss: 0.2173 - accuracy: 0.6667
Epoch 6/20
4/4 [==============================] - 0s 1ms/step - loss: 0.2169 - accuracy:

In [83]:
bestLoss = 0
bestAcc = 0
bestPosition = 0
position = 0

for model in listaModelos:
    print(f"- VALIDATION LOSS: {model[1][0]}\n- VALIDATION ACCURACY: {model[1][1]}\n\n")
    
    if model[1][1] > bestAcc:
        bestAcc = model[1][1]
        bestPosition = position

    position += 1
    
listaModelos[bestPosition][0].save('bestModelMeanSquaredError', save_format = 'tf')
listaModelos[bestPosition][0].save_weights('bestModelMeanSquaredErrorWeights', save_format = 'tf')

- VALIDATION LOSS: 0.1892823576927185
- VALIDATION ACCURACY: 0.6666666865348816


- VALIDATION LOSS: 0.18723948299884796
- VALIDATION ACCURACY: 0.6666666865348816


- VALIDATION LOSS: 0.16357070207595825
- VALIDATION ACCURACY: 0.6666666865348816


- VALIDATION LOSS: 0.21336695551872253
- VALIDATION ACCURACY: 0.4000000059604645


- VALIDATION LOSS: 0.2102479189634323
- VALIDATION ACCURACY: 0.6666666865348816


- VALIDATION LOSS: 0.2094782292842865
- VALIDATION ACCURACY: 0.6666666865348816


INFO:tensorflow:Assets written to: bestModelMeanSquaredError\assets


In [84]:
lossesModelos = ['binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy', 
                   'binary_crossentropy', 'binary_crossentropy', 'binary_crossentropy']

listaModelos = []

for i in range(0, len(neuronasOcultaModelos)):
    
    print(f"MODELO NUMERO {i+1}:\n")
    
    listaModelos.append(crearModelo(dataX, dataYC, dataXV, dataYVC, neuronasOcultaModelos[i], neuronasSalidaModelos[i],
               activacionesOcultaModelos[i], activacionesSalidaModelos[i], 
               epochsModelos[i], batchSizeModelos[i], lossesModelos[i]))

MODELO NUMERO 1:

MODELO:
	- Capa oculta (19, relu)
	- Capa salida (3, softmax)
	- Epochs: 15
	- Batch_size: 30
	- Loss metric: binary_crossentropy


Epoch 1/15
4/4 [==============================] - 0s 1ms/step - loss: 0.7144 - accuracy: 0.3333
Epoch 2/15
4/4 [==============================] - 0s 1ms/step - loss: 0.7090 - accuracy: 0.3333
Epoch 3/15
4/4 [==============================] - 0s 2ms/step - loss: 0.7038 - accuracy: 0.3333
Epoch 4/15
4/4 [==============================] - 0s 2ms/step - loss: 0.6988 - accuracy: 0.3333
Epoch 5/15
4/4 [==============================] - 0s 2ms/step - loss: 0.6940 - accuracy: 0.3333
Epoch 6/15
4/4 [==============================] - 0s 1ms/step - loss: 0.6896 - accuracy: 0.3333
Epoch 7/15
4/4 [==============================] - 0s 1ms/step - loss: 0.6853 - accuracy: 0.3333
Epoch 8/15
4/4 [==============================] - 0s 2ms/step - loss: 0.6813 - accuracy: 0.3333
Epoch 9/15
4/4 [==============================] - 0s 1ms/step - loss: 0.6775 - acc

4/4 [==============================] - 0s 2ms/step - loss: 0.6335 - accuracy: 0.3333
Epoch 15/15
1/1 [==============================] - 0s 95ms/step - loss: 0.6299 - accuracy: 0.3333

Resultados de la validacion
	- Error: 0.6299314498901367
	- Precision: 33.33333432674408%


MODELO NUMERO 5:

MODELO:
	- Capa oculta (19, sigmoid)
	- Capa salida (3, softmax)
	- Epochs: 20
	- Batch_size: 30
	- Loss metric: binary_crossentropy


Epoch 1/20
4/4 [==============================] - 0s 2ms/step - loss: 0.8643 - accuracy: 0.3333
Epoch 2/20
4/4 [==============================] - 0s 1ms/step - loss: 0.8515 - accuracy: 0.3333
Epoch 3/20
4/4 [==============================] - 0s 2ms/step - loss: 0.8391 - accuracy: 0.3333
Epoch 4/20
4/4 [==============================] - 0s 1000us/step - loss: 0.8269 - accuracy: 0.3333
Epoch 5/20
4/4 [==============================] - 0s 2ms/step - loss: 0.8150 - accuracy: 0.3333
Epoch 6/20
4/4 [==============================] - 0s 1000us/step - loss: 0.8042 - accura

In [87]:
bestLoss = 0
bestAcc = 0
bestPosition = 0
position = 0

for model in listaModelos:
    print(f"- VALIDATION LOSS: {model[1][0]}\n- VALIDATION ACCURACY: {model[1][1]}\n\n")
    
    if model[1][1] > bestAcc:
        bestAcc = model[1][1]
        bestPosition = position
    
    position += 1
    
listaModelos[bestPosition][0].save('bestModelBinaryCrossentropy', save_format = 'tf')
listaModelos[bestPosition][0].save_weights('bestModelBinaryCrossentropyWeights', save_format = 'tf')

- VALIDATION LOSS: 0.657522976398468
- VALIDATION ACCURACY: 0.3333333432674408


- VALIDATION LOSS: 0.6098605990409851
- VALIDATION ACCURACY: 0.6333333253860474


- VALIDATION LOSS: 0.5666093230247498
- VALIDATION ACCURACY: 0.6666666865348816


- VALIDATION LOSS: 0.6299314498901367
- VALIDATION ACCURACY: 0.3333333432674408


- VALIDATION LOSS: 0.6905402541160583
- VALIDATION ACCURACY: 0.3333333432674408


- VALIDATION LOSS: 0.6294620037078857
- VALIDATION ACCURACY: 0.5666666626930237


INFO:tensorflow:Assets written to: bestModelBinaryCrossentropy\assets


In [86]:
#resultadosPruebas = model.predict(dataXV, verbose=1)

"""predicciones = ['Setosa', 'Versicolor', 'Virginica']
cantidadFallos = 0

# Se calcula el error cuadratico medio
errorSalidas = 0
errorPatrones = 0
errorCMedio = 0

for i in range(0, len(resultadosPruebas)):
    print(f"Registro {i+1}:\n\t- Salidas obtenidas", end="")
    for j in range(0, len(resultadosPruebas[i])):
        print(f"\n\t\t- Neurona {j+1}: {resultadosPruebas[i][j]} ({predicciones[round(resultadosPruebas[i][j])]})", end="")
        errorSalidas += pow((dataYVC[i][j] - resultadosPruebas[i][j]), 2)
        
    if(type(dataYC[i]) != np.float64):
        print(f"\n\t- Salida esperada: {dataYVC[i][j]} ({predicciones[int(dataYVC[i][j])]})\n")
    else:
        print(f"\n\t- Salida esperada: {dataYVC[i]} ({predicciones[int(dataYVC[i])]})\n")
    
    errorPatrones += errorSalidas
    errorSalidas = 0
    
    
    if(type(dataYC[i]) != np.float64):
        if(mean(resultadosPruebas[i]) > 0.5 and dataYVC[i][j] == 0):
            cantidadFallos += 1
        if(mean(resultadosPruebas[i]) < 0.5 and dataYVC[i][j] == 1):
            cantidadFallos += 1
    else:
        if(mean(resultadosPruebas[i]) > 0.5 and dataYVC[i] == 0):
            cantidadFallos += 1
        if(mean(resultadosPruebas[i]) < 0.5 and dataYVC[i] == 1):
            cantidadFallos += 1
            
errorCMedio = (1/(2*len(resultadosPruebas)))*errorPatrones
        
print(f"La precision de las pruebas es del {round(((9-cantidadFallos)*100/9), 2)}%")
print(f"El error cometido es: {errorCMedio}")"""

'predicciones = [\'Setosa\', \'Versicolor\', \'Virginica\']\ncantidadFallos = 0\n\n# Se calcula el error cuadratico medio\nerrorSalidas = 0\nerrorPatrones = 0\nerrorCMedio = 0\n\nfor i in range(0, len(resultadosPruebas)):\n    print(f"Registro {i+1}:\n\t- Salidas obtenidas", end="")\n    for j in range(0, len(resultadosPruebas[i])):\n        print(f"\n\t\t- Neurona {j+1}: {resultadosPruebas[i][j]} ({predicciones[round(resultadosPruebas[i][j])]})", end="")\n        errorSalidas += pow((dataYVC[i][j] - resultadosPruebas[i][j]), 2)\n        \n    if(type(dataYC[i]) != np.float64):\n        print(f"\n\t- Salida esperada: {dataYVC[i][j]} ({predicciones[int(dataYVC[i][j])]})\n")\n    else:\n        print(f"\n\t- Salida esperada: {dataYVC[i]} ({predicciones[int(dataYVC[i])]})\n")\n    \n    errorPatrones += errorSalidas\n    errorSalidas = 0\n    \n    \n    if(type(dataYC[i]) != np.float64):\n        if(mean(resultadosPruebas[i]) > 0.5 and dataYVC[i][j] == 0):\n            cantidadFallos += 